# Apuntes Aprendizaje Reforzado

Fuente: https://www.youtube.com/watch?v=Mut_u40Sqz4

In [1]:
#!pip install stable-baselines3[extra]
#!pip install gym

In [2]:
import os
import gym
from stable_baselines3 import PPO
#Esto es el el algoritmo de aprendizaje que vamos a utilizar, aunque hay muchos diferentes
# y puede que este no sea el que nosotros necesitemos
from stable_baselines3.common.vec_env import DummyVecEnv
# DummyVecEnv nos sirve para vectorizar nuestros entornos y es maravilloso porque nos
# posibilita hacer muchas simulaciones en un vector, lo que acelera el proceso de manera extrema
from stable_baselines3.common.evaluation import evaluate_policy
# Esta libraria es para evaluar modelos

## Cargar Entorno

In [3]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)
# Vamos a trabajar con un problema de RL que consiste en mantener una barra vertical sin que caiga
# La barra se encuentra sobre una plataforma móvil
# El problema con el que trabajamos y cómo este se describe es lo que llamamos el entorno

C:\Users\jpeir\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [4]:
episodes = 5
for ep in range(episodes):
    state = env.reset()
    # Reinicia el entorno
    done = False
    # Estado del entorno
    score = 0
    # Recompensa/Puntuación
    
    while not done:
        env.render()
        # Muestra una interfaz del problema
        action = env.action_space.sample()
        # Realiza una acción aleatoria
        # En este problema, como tal y como se define en el entorno, solo se pueden hacer 2 posibles opciones, izquierda o derecha
        # Así que action_space es el espacio de posibles acciones, que como además son acciones discretas el objeto se guarda como Discrete(2)
        # Luego, cuando hacemos sample tomamos una de las 2 posibles acciones aleatoriamente
        n_state, reward, done, trunc, info = env.step(action)
        # Ejecutamos la accion elegida con step
        # Devuelve el estado al que pasamos (la definición del estado)
        # La recompensa que adquiere en ese paso
        # Si la mision se ha completado
        # Si la mision ha podido completarse en menos del maximo de pasos posibles
        # Informacion adicional
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:35.0
Episode:1 Score:19.0
Episode:2 Score:13.0
Episode:3 Score:14.0
Episode:4 Score:14.0


C:\Users\jpeir\anaconda3\lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(
C:\Users\jpeir\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
 env.reset()
env.action_space.sample()
env.step(action)

(array([ 0.0237855 ,  0.24117677,  0.03489403, -0.3260484 ], dtype=float32),
 1.0,
 False,
 False,
 {})

## Entrenar modelo

In [9]:
log_path = os.path.join("Entrenamiento", "Logs")
# Es muy importante que los directorios estén creados antes de hacer nada
# Aquí hemos creado una carpeta llamada Entrenamiento, dentro la cual hay una llamada Logs para guardar progreso
# Y otra Saved para guardar modelos entrenados
log_path

'Entrenamiento\\Logs'

In [11]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# Con esta linea creamos un entorno vectorizado, lo que nos permite paralelizar nuestro entorno y hacer muchos entrenamientos en paralelo
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)
# Aquí estamos creando el modelo con el algoritmo que hemos importado desde el principio y que decidimos que seria el que utilizaremos
# Hay muchos más algoritmos, pero para este ejemplo hemos preferido usar este
# El primer parametro es la política que utilizaremos, MlpPolicy viene de Multi-Layer Perceptron Policy
# Hay 3 posibles politicas de stable_baselines3: MlpPolicy, CnnPolicy, MultiInputPolicy
# Segundo atributo el entorno, (que ya hemos vectorizaodo antes)
# Luego verbose, porque queremos ver lo que ocurre, y la ruta de guardado, creada antes

C:\Users\jpeir\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [12]:
#PPO??
# Hay chorrocientos atributos más

In [14]:
model.learn(total_timesteps = 20000)
# Y voilá, así se entrena un modelín, no es dificil
# EL resultado del entrenamiento se guarda en una carpeta
# Cada vez que se ejecuta el bicho se crea una carpeta con lo que ha ido aprendiendo

Logging to Entrenamiento\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 645  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 385          |
|    iterations           | 2            |
|    time_elapsed         | 10           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0075981338 |
|    clip_fraction        | 0.104        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.556       |
|    explained_variance   | 0.814        |
|    learning_rate        | 0.0003       |
|    loss                 | 1.06         |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.0135      |
|    value_loss           | 6.93         |
-----------------------

## Guardar y recargar un modelo

In [15]:
PPO_Path = os.path.join("Entrenamiento", "Saved Models", "PPO_CartPole")
# Creamos una carpeta para guardar modelos y reejecutarlos, no hace falta que ya exista, sino que la creará
# La creará como un zip
model.save(PPO_Path)

In [19]:
# Si quisieramos, ahora ya tenemos el modelo guardado así que podemos cargarlo de nuevo
# Podemos eliminar el modelo que tenemos en memoria
del model # Eliminar modelo
# model.learn(total_timesteps = 20000) # Esto daría error porque la variable ya no existe

In [20]:
# Y lo podemos importar de la carpeta que teniamos guardada
model = PPO.load(PPO_Path, env = env)

## Evaluar modelo

In [23]:
# Usamos la función que hemos importado al principio para evaluar modelos
evaluate_policy(model, env, n_eval_episodes = 10, render=True)
# Podemos elegir el número de episodios a evaluar
# Y con render elegimos si queremos verlo ejecutar o no

In [ ]:
env.close() # Y cerramos

## Testear

In [27]:
# Ya hemos evaluado el modelo, pero ahora, la pregunta realmente importante es, qué ocurrirá si cambia el entorno
# Es el motivo por el que hacemos todo esto
# Tomamos la idea del código del principio

episodes = 5 # Numero episodios
for ep in range(episodes):
    obs = env.reset()
    # Reiniciamos el entorno, para ver qué pasa si cambia
    # Guardamos el entorno el una variable
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        # Esto es casi lo único que cambia respecto al principio
        # Ahora en lugar de acciones aleatorias va a predecir las acciones
        # Tenemos que pasarle el entorno para que prediga
        obs, reward, done, info = env.step(action)
        # Y el entorno se tiene que actualizar en cada iteracion
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:[200.]
Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]


In [ ]:
env.close()